# [Optional] Run the image classifier on inputs from webcam/ JS canvas (in realtime)  

This is an optional notebook for this week, run this only if you're interested in it.

This notebook allows you to use real-time data from webcam or Javascript canvas as inputs for the image classifier that we trained in the `02_image_classifier_pytorch.ipynb`  


We will use a Python library called [Flask](https://flask.palletsprojects.com/en/3.0.x/). It allows you to create web applications in your local Python environment. The advantage of using Flask is that you can build web interfaces for your Python programs and use Javascript to process media inputs such as webcam, audio, or JS painting canvas. 


READ BEFORE PROCEEDING:

1. This notebook only works if it's running on your local machine, this means - **it doesn't work on Google Colab** or other cloud platforms.  
   It also doesn't work if you're remotely connecting to devices in CCI.  
2. As we mentioned in the first lecture, Python is not good at processing real-time data. So, this approach is definately not the best practice, but it's good enough for prototype and demonstration purposes - use it with caution.

Have fun!

## Step 0 - Download and installation (do this only once)

This is a GitHub repository for real-time bi-directional communication between Python server and front-end client.  
Created by Jasper https://github.com/jasper-zheng/realtime-flask-notebook , feel free to reuse it


In [1]:
!git clone https://github.com/jasper-zheng/realtime-flask-notebook.git

Cloning into 'realtime-flask-notebook'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 46 (delta 11), reused 41 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 168.08 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
!pip install flask Flask-SocketIO Werkzeug

## Step 1 - Load the trained model  

Make sure you have finished the `02_image_classifier_pytorch.ipynb` notebook, and have the trained model saved as `model_final.pt`

Again, before we start, let's config the device PyTorch is using

In [3]:
import torch

device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

elif torch.backends.mps.is_available():
    device = "mps"

print(f'Using device: {device}')

Using device: mps


In [4]:
from src.model import ConvNeuralNetwork
import torch

In [ ]:
img_channel = _______ # number of colour channels of inputs (img_channel = 1 for greyscale, img_channel = 3 for RGB).
img_resolution = _______ # width and height of input images, this should match the model you trained (e.g. img_resolution = 64 for 64x64 inputs)
num_classes = _______ # how many categories the model is classifying (e.g. num_classes = 3 for 3 classes)

# make sure the parameters are the same as when the model is created
model = ConvNeuralNetwork(img_channel = img_channel, 
                          img_resolution = img_resolution,
                          num_classes = num_classes)

# load the saved model, make sure the path is correct
model.load_state_dict(torch.load('model_final.pt'))

model.to(device)
model.eval()

## Step 2 - Set up the web application  

Config the web application using [Flask](https://flask.palletsprojects.com/en/3.0.x/api/) and [Socket.IO](https://socket.io/docs/v4/).  
We can skip over some technical details of how this works, but as an overview, they're going to help us set up a **server** that runs our classifier model, and a **client** that runs the Javascript interface. The client sends a frame of input image to the server, the server uses the classifier model to predict a class, and sends the prediction back to the client.

<img src="./src/graphics/client_server.jpg" width="600px"></img>  

First, let's import

In [ ]:
import sys

base_dir = 'realtime-flask-notebook'
sys.path.append(f'{base_dir}')

from FlaskProcessor import Processor

import os, sys
import logging

from flask import Flask, render_template
from flask_socketio import SocketIO, emit

Then, config the base directory that the web app is running

In [6]:
template_dir = os.path.abspath(f'{base_dir}/templates')
static_dir = os.path.abspath(f'{base_dir}/static')

app = Flask(__name__, static_folder=static_dir, template_folder=template_dir)
socketio = SocketIO(app)
processor = Processor(model, device = device, colour_channels = img_channel, img_resolution = img_resolution)

@socketio.on('packet_from_js', namespace='/demo')
def packet_from_js(packet_from_js):
    img = packet_from_js.split(",")[1]
    processor.enqueue_input(img)
    class_idx = processor.get_frame()
    emit('packet_from_py', {'class_name': class_idx}, namespace='/demo')

## Step 3 - Activate the web app and go to its URL

**Follow option 1 or option 2:**

### Option 1: Inputs from Webcam  

If you chose to train the Pinterest image classifier, you might want to go with this option, it will use your webcam as input for the model

In [ ]:
@app.route('/')
def index():
    return render_template('webcam.html')

# enter the names of cateories you have
processor.set_class_name(['_____', '_____', '_____', .....])

In [ ]:
port = 5005
print(f'open your browser and go to http://127.0.0.1:{port}/ \n')

socketio.run(app, port=port, allow_unsafe_werkzeug=True)

### Option 2: Sketch via mouse  

If you chose to train the sketch classifier, you might want to go with this option, it will open a Javascript canvas where you can use your mouse to draw sketches.

In [ ]:
@app.route('/')
def index():
    return render_template('sketch.html')

# enter the names of cateories you have
processor.set_class_name(['apple', 'basketball', 'book', 'cat', 'duck', 'fish', 'flower', 'mouth', 'mushroom', 'whale'])

In [ ]:
port = 5005
print(f'open your browser and go to http://127.0.0.1:{port}/ \n')

socketio.run(app, port=port, allow_unsafe_werkzeug=True)